In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pandas_datareader import data

In [ ]:
# ticker
tickers = 'BOX'

train_start_date = '2016-03-31'
train_end_date = '2018-03-31'

train_data = data.DataReader(tickers, 'iex', train_start_date, train_end_date)
#panel_data = panel_data.rename(columns={'close': tickers})
train_close = train_data["close"]

validation_start_date = '2018-03-31'
validation_end_date = '2019-03-31'

validation_data = data.DataReader(tickers, 'iex', validation_start_date, validation_end_date)
#panel_data = panel_data.rename(columns={'close': tickers})
validation_close = validation_data["close"]




In [ ]:
def generate_signals(close, wind, top_sigma, bottom_sigma):
        """Returns the DataFrame of symbols containing the signals
        to go long, short or hold (1, -1 or 0)."""
        signals = pd.DataFrame(index=close.index)
        comp = pd.DataFrame(index=close.index)
        comp['long_comp'] = 0.0
        comp['short_comp'] = 0.0
        signals['signals']= 0.0
        
        # Create upper and lower band
        rstd = close.rolling(window=wind).std()

        
        sma = close.rolling(window=wind).mean()
        upper_band = sma + top_sigma * rstd
        lower_band = sma - bottom_sigma * rstd

        signals['mavg'] = sma
        comp['upper'] = upper_band
        comp['lower'] = lower_band
        
        

        comp['long_comp'][wind:] = np.where(close[wind:] 
            < comp['lower'][wind:], 1.0, 0.0)  
        comp['short_comp'][wind:] = np.where(close[wind:] 
            > comp['upper'][wind:], 1.0, 0.0)  
         
        

        for i in range(1, len(signals['mavg'][wind:])):
            if comp['long_comp'][i]==1 and comp['long_comp'][i-1]==0:
                signals['signals'][i] = 1
            if comp['short_comp'][i]==1 and comp['short_comp'][i-1]==0:
                signals['signals'][i] = -1
                



        return signals

In [ ]:
def port(sig, close, per_buy):
    aum = 1000
    cash = 1000
    shares = 0.0
    ret = 0.0
    for i in range(0, len(sig['signals'])):
        if sig['signals'][i] == 1.0:
            num_shares = per_buy*aum/close[i]
            cash -= per_buy*aum
            shares+= num_shares
            
        elif sig['signals'][i] == -1.0:
            num_shares = per_buy*aum/close[i]
            cash += per_buy*aum
            shares-= num_shares
            
        #print("Cash:", cash, "shares:", shares)
        
    #print(shares*close[-1]+cash)
    ret = ((shares*close[-1]+cash-aum))/aum
    
    
    
    return [cash, shares, ret]
    
            

In [ ]:
def calc_ret(close, wind, top_sigma, bottom_sigma, per_buy):
    signals=generate_signals(close, wind, top_sigma, bottom_sigma)
    return port(signals, close, per_buy)[2]


In [ ]:
#calc_ret(train_close, 6, 0.11787, 0.703172, 1000, 0.388447)

In [ ]:
#generate a training set with n entries
def gen_train(df,aum,n):
    
    train_set = pd.DataFrame(index=np.arange(0,n))
    
    train_set['windows'] = np.random.randint(1, 100, n)
    train_set['top_sigmas'] = np.random.uniform(0.01, 3, n)
    train_set['bottom_sigmas'] = np.random.uniform(0.01, 3, n)
    train_set['per_buys'] = np.random.uniform(0, 0.3, n)
    
    

    train_set["returns"] = 0.0
    for i in range(0,n):
        train_set['returns'][i]=calc_ret(df, train_set['windows'][i], train_set['top_sigmas'][i], train_set['bottom_sigmas'], aum, train_set['per_buys'][i])
    return train_set

In [ ]:
#gen_train(train_close, 10000, 50)

In [ ]:
def RSI(series, period):
    delta = series.diff().dropna()
    up = delta * 0
    down = up.copy()
    up[delta > 0] = delta[delta > 0]
    down[delta < 0] = -delta[delta<0]
    up[up.index[period-1]] = np.mean( up[:period])
    up = up.drop(up.index[:(-period-1)])
    down[down.index[period-1]] = np.mean(down[:period])
    down = down.drop(down.index[:(period-1)])
    rs = pd.DataFrame.ewm (up, com = period-1, adjust = False)
    rs = rs.divide(pd.DataFrame.ewm (down, com = period-1, adjust = False))
    return 100 - 100/(1+rs)
    

In [ ]:
def get_stock(stock,start,end):
     return data.DataReader(stock,'google',start,end)['Close']

In [ ]:
df = train_data
df['RSI'] = RSI(df['close'], 14)
df.tail()